<a href="https://colab.research.google.com/github/MengOonLee/LLM/blob/main/References/LangChain/ipynb/Academy/LangChain/Quickstart/L1_fast_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fast Agent - Build a SQL Agent fast!

<img src="https://github.com/MengOonLee/LLM/blob/main/References/LangChain/ipynb/Academy/LangChain/Quickstart/assets/LC_L1_Top.png?raw=true" align="left" width="500">

## Setup

Load and/or check for needed environmental variables

In [ ]:
%%bash
pip install --no-cache-dir -qU pip
pip install --no-cache-dir -qU \
    langchain langgraph langchain-core langchain-community \
    langchain-huggingface

Initialize LLM model

In [1]:
import warnings
warnings.filterwarnings("ignore")
import langchain_huggingface

# repo_id = "Qwen/Qwen2.5-7B-Instruct"
repo_id = "Qwen/Qwen3-4B-Thinking-2507-FP8"
llm = langchain_huggingface.HuggingFaceEndpoint(repo_id=repo_id,
    temperature=0.1)
llm = langchain_huggingface.ChatHuggingFace(llm=llm)

In [2]:
from langchain_community import utilities

db = utilities.SQLDatabase.from_uri("sqlite:///Chinook.db")

Define the runtime context to provide the agent and tools with access to the database.

In [3]:
from dataclasses import dataclass

from langchain_community import utilities

# define context structure to support dependency injection
@dataclass
class RuntimeContext:
    db: utilities.SQLDatabase

<b>⚠️ Security Note:</b> This demo does not include a filter on LLM-generated commands. In production, you would want to limit the scope of LLM-generated commands. ⚠️   
This tool will connect to the database. Note the use of `get_runtime` to access the graph **runtime context**.

In [4]:
from langchain_core.tools import tool
from langgraph import runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtimes = runtime.get_runtime(RuntimeContext)
    db = runtimes.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

Add a system prompt to define your agents behavior.

In [5]:
SYSTEM_PROMPT = """
You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows of output unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

Create your agent! Add a model, tools, a prompt, and the runtime access, and go!  You can choose many agents from our [integrations](https://docs.langchain.com/oss/python/integrations/providers) list.

In [1]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

ImportError: cannot import name 'create_agent' from 'langchain.agents' (/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/langchain/agents/__init__.py)

Here's a display of the agent ReAct Loop.

In [ ]:
from IPython import display

display.display(display.Image(agent.get_graph(xray=True).draw_mermaid_png()))

Run some queries. Notice:
- The agent does not have the database schema and will need to discover it independently.
- The agent may make mistakes! By returning error messages, the agent can self-correct its queries.
- Notice you invoke the agent with `agent.stream`.
    - This command and the `pretty_print` display the **messages** that communicate information between the model and the tools.
- Notice the agent doesn't remember the schema between invocations... More on this later!

In [ ]:
question = """
Which table has the largest number of entries? The tables name are:
 - Artist
 - Customer
 - playlistTrack
"""

for step in agent.stream(
    {"messages": question},
    context=RuntimeContext(db=db),
    stream_mode="values"
):
    step["messages"][-1].pretty_print()

In [ ]:
question = "Which genre on average has the longest tracks?"

for step in agent.stream(
    {"messages": question},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
question = "Please list all of the tables"

for step in agent.stream(
    {"messages": question},
    context={"db": db},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

**Create your own query here!**  Add some questions of your own.

In [ ]:
question = "TRY YOUR OWN QUERY HERE"

for step in agent.stream(
    {"messages": question},
    context={"db": db},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

### Let's try this Studio